In [63]:
import pandas as pd
import numpy as np
import re
import string
import os
import json
from bidict import bidict
import pickle
import json
import random
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem.snowball import SnowballStemmer
from math import ceil
from pprint import pprint
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB, BernoulliNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn.metrics import confusion_matrix, classification_report
import itertools

In [2]:
with open("/data/rali7/Tmp/solimanz/data/datasets/top550/train_ids.pkl", "rb")as f:
    train_ids = pickle.load(f)
with open("/data/rali7/Tmp/solimanz/data/datasets/top550/test_ids.pkl", "rb")as f:
    test_ids = pickle.load(f)
with open("/data/rali7/Tmp/solimanz/data/datasets/top550/jobid/data.json", "r") as f:
    data = json.load(f)

In [3]:
def multiomial_nb(X_train, train_targets, X_test, test_targets):
    # Train
    multi_nb = MultinomialNB()
    print("Training Multinomial Naive Bayes...")
    multi_nb.fit(X_train, train_targets)

    # Test
    print("Running trained model on test dataset")
    predicted = multi_nb.predict(X_test)
    acc = np.mean(predicted == test_targets)

    print("Model Accuracy: " + str(acc))
    
    return multi_nb
    
def bernoulli_nb(X_train, train_targets, X_test, test_targets):
    # Train
    nb = BernoulliNB()
    print("Training Multinomial Naive Bayes...")
    nb.fit(X_train, train_targets)

    # Test
    print("Running trained model on test dataset")
    predicted = multi_nb.predict(X_test)
    acc = np.mean(predicted == test_targets)

    print("Model Accuracy: " + str(acc))
    
    return nb
    
def run_model(model, X_train, train_targets, X_test, test_targets):
    # Train
    print(f"Training Multinomial Naive Bayes...")
    model.fit(X_train, train_targets)

    # Test
    print("Running trained model on test dataset")
    predicted = model.predict(X_test)
    acc = np.mean(predicted == test_targets)

    print("Model Accuracy: " + str(acc))
    
    return model

def make_X(sequences, vectorizer, vocab_size=None, counts=False, text=True):
    if text:
        X = vectorizer.transform(sequences)
    else:
        X = np.zeros((len(sequences), vocab_size), dtype=np.int16)
        for i, seq in enumerate(sequences):
            for elem in seq:
                if counts:
                    X[i][elem] += 1
                else:
                    X[i][elem] = 1

    return X

In [72]:
word_tokenize("allo/bye")

['allo/bye']

In [46]:
def get_data_sets(json_path, tokenize_titles=True, use_stemmer=True):
    
    with open(json_path, 'r') as f:
        data = json.load(f)
    
    title_id = bidict(data["title_to_id"])
    
    stemmer = SnowballStemmer('english')
    
    if tokenize_titles:
        train_seq = [[title_id.inv[i] for i in title_seq[:-1]] for title_seq in data["train_data"]]
        test_seq = [[title_id.inv[i] for i in title_seq[:-1]] for title_seq in data["test_data"]]
    else:
        train_seq = [[re.sub(r"[\W\s]", "_", title_id.inv[i]) for i in title_seq[:-1]] for title_seq in data["train_data"]]
        test_seq = [[re.sub(r"[\W\s]", "_", title_id.inv[i]) for i in title_seq[:-1]] for title_seq in data["test_data"]]
    
    train_targets = [seq[-1] for seq in data["train_data"]]
    test_targets = [seq[-1] for seq in data["test_data"]]
    
    if tokenize_titles:
        train_text = [" ".join(title_seq).replace("_", " ") for title_seq in train_seq]  
        test_text = [" ".join(title_seq).replace("_", " ") for title_seq in test_seq]
    else:
        train_text = [" ".join(title_seq) for title_seq in train_seq]  
        test_text = [" ".join(title_seq) for title_seq in test_seq]
        
    return train_text, test_text

In [47]:
reduced7000_path = "/data/rali7/Tmp/solimanz/data/datasets/reduced7000/jobid/data.json"   
top550_path = "/data/rali7/Tmp/solimanz/data/datasets/top550/jobid/data.json"   

In [48]:
train550, test550 = get_data_sets(top550_path, False)

In [65]:
stemmer = SnowballStemmer('english')
lemmer = WordNetLemmatizer()

In [70]:
stemmer.stem('software')

'softwar'

In [62]:
lemmer.lemmatize('engineer')

'engineer'

In [23]:
cv = CountVectorizer(binary=False)
cv.fit(all_t)

ValueError: empty vocabulary; perhaps the documents only contain stop words

In [15]:
cv.vocabulary_

{'am': 0,
 'and': 1,
 'animal': 2,
 'ape': 3,
 'great': 4,
 'hello_world': 5,
 'helloworld': 6,
 'human': 7}

In [23]:
X_train = make_X(train_text, cv, counts=True, text=True) 
X_test = make_X(test_text, cv, counts=True, text=True)

In [24]:
multi_nb = MultinomialNB()
multi_nb.fit(X_train, train_targets)

MultinomialNB(alpha=1.0, class_prior=None, fit_prior=True)

In [25]:
nb = BernoulliNB()
nb.fit(X_train, train_targets)

BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None, fit_prior=True)

In [26]:
def top_k_acc(model, k=1):
    preds = model.predict_proba(X_test)
    sorted_args = (-preds).argsort(axis=1)[:,:k]
    tt = np.tile(test_targets, (k,1)).T
    acc = np.mean(np.sum(sorted_args == tt, axis=1))
    return acc

In [ ]:
print(f"acc: {top_k_acc(multi_nb, k=1)*100:.2f}")
print(f"top 2: {top_k_acc(multi_nb, k=2)*100:.2f}")
print(f"top 3: {top_k_acc(multi_nb, k=3)*100:.2f}")
print(f"top 4: {top_k_acc(multi_nb, k=4)*100:.2f}")
print(f"top 5: {top_k_acc(multi_nb, k=5)*100:.2f}")

acc: 15.09
top 2: 22.07
top 3: 26.50
top 4: 29.77
top 5: 32.38


In [ ]:
print(f"acc: {top_k_acc(nb, k=1)*100:.2f}")
print(f"top 2: {top_k_acc(nb, k=2)*100:.2f}")
print(f"top 3: {top_k_acc(nb, k=3)*100:.2f}")
print(f"top 4: {top_k_acc(nb, k=4)*100:.2f}")
print(f"top 5: {top_k_acc(nb, k=5)*100:.2f}")

acc: 13.04
top 2: 19.50
